In [1]:
# Instalacion de la libreria para acceso a mysql
%pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cargamos la libreria para conectarnos a MySQL
import mysql.connector


In [3]:
# Lo primero es crear un objeto de conexión con la info del servidor
cnx = mysql.connector.connect(
    host="ec2-35-93-190-108.us-west-2.compute.amazonaws.com",
    port=3306,
    user="data",
    password="khQ9--mp6LWq97iA",
    database="movielens"
)

In [4]:
# Creamos un cursor que es un objeto que nos permite hacer consultas y obtener respuestas
cursor = cnx.cursor()

In [5]:
# Enviamos una consulta SQL
cursor.execute('SELECT * FROM movies')

In [6]:
# Recogemos todas las filas de la consulta con fetchall() y las asignamos a result
result = cursor.fetchall()

In [7]:
# ¡¡¡¡¡  MUY IMPORTANTE !!!!!
# Cerrar el objeto de conexión
cursor.close()

True

In [8]:
# Veamos  que tipo de dato es result
type(result)

list

In [9]:
# Vemos que el elemento es una tupla
result[0]

(1, 'Toy Story (1995)', "Animation|Children's|Comedy")

In [10]:
# Cargamos la libreria pandas para pasar la info a un dataframe
import pandas as pd

movies = pd.DataFrame(result, columns=['movie_id', 'title', 'genres'])

movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [11]:
# Vamos a ver el contenido de nuestra BD
cursor = cnx.cursor() # Conectamos el cursor

cursor.execute("SHOW TABLES")
result = cursor.fetchall()
cursor.close()  # Cerramos  la conexión del cursor

print(result)

[('age_ranges',), ('movies',), ('occupations',), ('ratings',), ('users',)]


In [12]:
# Ahora vamos a obtener la información de cada tabla y la guardamos en DFs
cursor = cnx.cursor()   # Conectamos el cursor

# users
cursor.execute("SELECT * FROM users")
result = cursor.fetchall()
users = pd.DataFrame(result, columns=['user_id', 'gender', 'age', 'occupation', 'cp'])

# ratings
cursor.execute("SELECT * FROM ratings")
result = cursor.fetchall()
ratings = pd.DataFrame(result, columns=['user_id', 'movie_id', 'rating', 'timestamp'])

# age ranges
cursor.execute("SELECT * FROM age_ranges")
result = cursor.fetchall()
age_ranges = pd.DataFrame(result, columns=['age_id', 'range'])

# occupations
cursor.execute("SELECT * FROM occupations")
result = cursor.fetchall()
occupations = pd.DataFrame(result, columns=['occupation_id', 'description'])

cursor.close()  # Cerramos la conexión del cursor

True

# Merge()
- Vamos a unir nuestros df con esta función

In [13]:
# Antes de unir vamos a ver los contenidos de los df
users.head()

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [14]:
# Vemos el tamaño
users.shape

(6040, 5)

In [15]:
occupations.head()

,occupation_id,description
0,0,other or not specified
1,1,academic/educator
2,2,artist
3,3,clerical/admin
4,4,college/grad student


In [16]:
# Tamaño
occupations.shape

(21, 2)

In [17]:
# Lo primero es identificar los id y convertirlos a indices
users = users.set_index('user_id', drop= True)
occupations = occupations.set_index('occupation_id', drop= True)

In [18]:
users

,gender,age,occupation,cp
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455
...,...,...,...,...
6036,F,25,15,32603
6037,F,45,1,76006
6038,F,56,1,14706


In [19]:
occupations.head()

,description
occupation_id,
0,other or not specified
1,academic/educator
2,artist
3,clerical/admin
4,college/grad student


In [20]:
# Vamos a realizar la union de la siguiente forma
# users     --> primer df o de la izquierda
# occupations   --> segundo df o de la derecha
# left_on= 'occupation'  -->  usara la columna occupation de users
# right_index= True     --> usara el indice de occupations
# sort_index()      -->     realiza un ordenamiento por el indice
# head(20)      -->     muestra las 20 primeras filas
pd.merge(users, occupations, left_on= 'occupation', right_index= True).sort_index().head(20)


,gender,age,occupation,cp,description
user_id,,,,,
1,F,1,10,48067,K-12 student
2,M,56,16,70072,self-employed
3,M,25,15,55117,scientist
4,M,45,7,02460,executive/managerial
5,M,25,20,55455,writer
6,F,50,9,55117,homemaker
7,M,35,1,06810,academic/educator
8,M,25,12,11413,programmer
9,M,25,17,61614,technician/engineer


In [21]:
users_full = pd.merge(users, occupations, left_on= 'occupation', right_index= True).sort_index()

users_full.shape

(6040, 5)

In [22]:
# Le vamos a cambiar el nombre a algunas columnas
users_full = users_full.rename(columns={'description': 'occupation', 'occupation': 'occupation_id'})

users_full.head()

,gender,age,occupation_id,cp,occupation
user_id,,,,,
1,F,1,10,48067,K-12 student
2,M,56,16,70072,self-employed
3,M,25,15,55117,scientist
4,M,45,7,02460,executive/managerial
5,M,25,20,55455,writer


In [23]:
# Veamos age_ranges
age_ranges.head()

,age_id,range
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49


In [24]:
# Seteamos el indice con age_id
age_ranges = age_ranges.set_index('age_id', drop= True)

age_ranges.head()

,range
age_id,
1,Under 18
18,18-24
25,25-34
35,35-44
45,45-49


In [25]:
# Lo agregamos a users_full
users_full = pd.merge(users_full, age_ranges, left_on= 'age', right_index= True).sort_index()

users_full.head()

,gender,age,occupation_id,cp,occupation,range
user_id,,,,,,
1,F,1,10,48067,K-12 student,Under 18
2,M,56,16,70072,self-employed,56+
3,M,25,15,55117,scientist,25-34
4,M,45,7,02460,executive/managerial,45-49
5,M,25,20,55455,writer,25-34


In [26]:
# Renombramos columnas
users_full = users_full.rename(columns= {'age':'age_id', 'range': 'age_ranges'})

users_full.head()

,gender,age_id,occupation_id,cp,occupation,age_ranges
user_id,,,,,,
1,F,1,10,48067,K-12 student,Under 18
2,M,56,16,70072,self-employed,56+
3,M,25,15,55117,scientist,25-34
4,M,45,7,02460,executive/managerial,45-49
5,M,25,20,55455,writer,25-34


In [27]:
users_full.shape

(6040, 6)

# Groupby

In [28]:
# Vamos a usar agrupaciones, es algo diferente a los filtros y muy util
users_full.groupby('gender')

In [29]:
# Ya tenemos una agrupación, ahora agregamos una columna de información para la agrupación
users_full.groupby('gender')['occupation']

In [30]:
# Bien ¿ahora porque no tenemos la información?
# Al realizar la agrupación necesitamos informar que necesitamos hacer con el grupo de datos
# Para eso usamos las funciones de agregación
users_full.groupby('gender')['occupation'].size() # size nos da la cantidad de datos(cuenta nulos también)

gender
F    1709
M    4331
Name: occupation, dtype: int64

In [31]:
# Podemos obtener las cantidades de ocupaciones 
users_full.groupby('gender')['occupation'].value_counts() # Cuenta las repeticiones

gender  occupation            
F       college/grad student      234
        other or not specified    232
        academic/educator         209
        executive/managerial      139
        doctor/health care        102
        clerical/admin            100
        artist                     91
        homemaker                  89
        sales/marketing            79
        writer                     78
        K-12 student               66
        technician/engineer        52
        self-employed              51
        programmer                 50
        retired                    34
        customer service           31
        scientist                  28
        lawyer                     22
        unemployed                 15
        tradesman/craftsman         4
        farmer                      3
M       executive/managerial      540
        college/grad student      525
        other or not specified    479
        technician/engineer       450
        programmer 

In [32]:
# Agrupación de multiple información
users_full.groupby(['gender', 'age_ranges']).size()

gender  age_ranges
F       18-24          298
        25-34          558
        35-44          338
        45-49          189
        50-55          146
        56+            102
        Under 18        78
M       18-24          805
        25-34         1538
        35-44          855
        45-49          361
        50-55          350
        56+            278
        Under 18       144
dtype: int64

In [33]:
# Si queremos acceder a un elemento especifico de este resultado 
# lo guardamos y accedemos con localización por multiíndice
gender_age_range = users_full.groupby(['gender', 'age_ranges']).size()
gender_age_range.loc[('M','45-49')]

361

In [34]:
# Podemos filtrar ocupaciones por genero
gender_occupation_count = users_full.groupby('gender')['occupation'].value_counts()

# Separamos datos entre masculinos y femeninos
male_occupation_count = gender_occupation_count.loc['M']
female_occupation_count = gender_occupation_count.loc['F']

In [35]:
# Ocupaciones de hombres
print('Los puestos ocupados por hombres son:')
male_occupation_count

Los puestos ocupados por hombres son:


occupation
executive/managerial      540
college/grad student      525
other or not specified    479
technician/engineer       450
programmer                338
academic/educator         319
sales/marketing           223
writer                    203
self-employed             190
artist                    176
doctor/health care        134
K-12 student              129
scientist                 116
retired                   108
lawyer                    107
customer service           81
clerical/admin             73
tradesman/craftsman        66
unemployed                 57
farmer                     14
homemaker                   3
Name: count, dtype: int64

In [36]:
# Ocupaciones de mujeres
print('Los puestos ocupados por mujeres son:')
female_occupation_count

Los puestos ocupados por mujeres son:


occupation
college/grad student      234
other or not specified    232
academic/educator         209
executive/managerial      139
doctor/health care        102
clerical/admin            100
artist                     91
homemaker                  89
sales/marketing            79
writer                     78
K-12 student               66
technician/engineer        52
self-employed              51
programmer                 50
retired                    34
customer service           31
scientist                  28
lawyer                     22
unemployed                 15
tradesman/craftsman         4
farmer                      3
Name: count, dtype: int64

# Metodo agg() de pandas
- Podemos usar el método agg para pasar varias funciones de agregación

In [37]:
# Calcular cual es la categoría que mas se repite
users_full.groupby(['gender'])['occupation'].agg(pd.Series.mode)

gender
F    college/grad student
M    executive/managerial
Name: occupation, dtype: object

In [38]:
users_full.head(3)

,gender,age_id,occupation_id,cp,occupation,age_ranges
user_id,,,,,,
1,F,1,10,48067,K-12 student,Under 18
2,M,56,16,70072,self-employed,56+
3,M,25,15,55117,scientist,25-34


In [39]:
# Calculo de media, desviación estándar, mediana en columna age_id
users_full.groupby(['gender'])['age_id'].agg(['mean', 'std', 'median'])

,mean,std,median
gender,,,
F,30.859567,13.242564,25.0
M,30.552297,12.757110,25.0
